<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.26.0
Pandas:  2.1.1


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] No such file or directory: 'bikeshare.csv'

ANSWER:

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [11]:
#ANSWER:
bikes = pd.read_table('../../../IOD/DATA/bikeshare.csv', header = 0, sep=',')
print(bikes.head())


   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be dealt with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [ ]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)


In [4]:
df = pd.read_excel('../../../IOD/DATA/Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For example, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [5]:
#ANSWER - not sure what is 'unwanted' here

df = pd.read_excel('../../../IOD/DATA/Iris.xls', sheet_name = 'Data', usecols=[0,1,2,3,4])

df


,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length
0,1,0.2,1.4,3.5,5.1
1,1,0.2,1.4,3.0,4.9
2,1,0.2,1.3,3.2,4.7
3,1,0.2,1.5,3.1,4.6
4,1,0.2,1.4,3.6,5.0
...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7
146,3,1.9,5.0,2.5,6.3
147,3,2.0,5.2,3.0,6.5
148,3,2.3,5.4,3.4,6.2


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [6]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-12-19T09:31:41.182Z,39.6209,-119.6354,5.9,3.4,ml,13,72.6,0.06,0.192,...,2023-12-19T10:27:11.428Z,"6 km ESE of Spanish Springs, Nevada",earthquake,NaN,2.8,0.29,5,reviewed,nn,nn


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [7]:

url = 'https://www.ccra.com/airport-codes/'

df = pd.read_html(url)
df

[                City      Country Code
 0            Aalborg      Denmark  AAL
 1           Aalesund       Norway  AES
 2             Aarhus      Denmark  AAR
 3     Abbotsford, BC       Canada  YXX
 4     Abbotsford, BC       Canada  YXX
 ...              ...          ...  ...
 1574        Zanzibar     Tanzania  ZNZ
 1575        Zaragoza        Spain  ZAZ
 1576       Zhengzhou        China  CGO
 1577        Zhoushan        China  HSN
 1578          Zurich  Switzerland  ZRH
 
 [1579 rows x 3 columns]]

In [8]:
df[0]

,City,Country,Code
0,Aalborg,Denmark,AAL
1,Aalesund,Norway,AES
2,Aarhus,Denmark,AAR
3,"Abbotsford, BC",Canada,YXX
4,"Abbotsford, BC",Canada,YXX
...,...,...,...
1574,Zanzibar,Tanzania,ZNZ
1575,Zaragoza,Spain,ZAZ
1576,Zhengzhou,China,CGO
1577,Zhoushan,China,HSN


#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here are a few examples: https://pandas.pydata.org/docs/user_guide/io.html#io-read-xml

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example:

In [9]:
url = 'https://microsoftedge.github.io/Demos/json-dummy-data/64KB.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,name,language,id,bio,version
0,Adeel Solangi,Sindhi,V59OF92YF627HFY0,Donec lobortis eleifend condimentum. Cras dict...,6.10
1,Afzal Ghaffar,Sindhi,ENTOCR13RSCLZ6KU,"Aliquam sollicitudin ante ligula, eget malesua...",1.88
2,Aamir Solangi,Sindhi,IAKPO3R4761JDRVG,Vestibulum pharetra libero et velit gravida eu...,7.27
3,Abla Dilmurat,Uyghur,5ZVOEPMJUI4MB4EN,Donec lobortis eleifend condimentum. Morbi ac ...,2.53
4,Adil Eli,Uyghur,6VTI8X6LL0MMPJCC,"Vivamus id faucibus velit, id posuere leo. Mor...",6.49


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [12]:
#ANSWER
bikes.shape

(17379, 17)

What are the column names in `bikes`?

In [13]:
#ANSWER
bikes.columns


Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [14]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [15]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this.
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [16]:
#ANSWER
type(bikes['season']) == type(bikes[['season']])

False

How would we use object notation to show the first 4 rows of `atemp`?

In [ ]:
#ANSWER
#bikes['atemp'].head(4)

bikes['atemp'][:4]

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ):

In [20]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: "None of [Index([0], dtype='int32')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [24]:
#ANSWER
bikes.iloc[:1]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [26]:
#ANSWER
bikes.iloc[:,1:2]

,dteday
0,2011-01-01
1,2011-01-01
2,2011-01-01
3,2011-01-01
4,2011-01-01
...,...
17374,2012-12-31
17375,2012-12-31
17376,2012-12-31
17377,2012-12-31


## Handling Missing Values

What is the Pandas `isnull` function for?

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [27]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [28]:
#ANSWER
bikes.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [29]:
#ANSWER
bikes.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: not a valid number

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [ ]:
#ANSWER
def divide(dividend, divisor):
    if divisor == 0:
        quotient = np.nan
    else:
        quotient = dividend / divisor
    return (quotient)

print(divide(1, 0))

Apply the Pandas `isna` function to the following data objects:

In [ ]:
x = 2.3
y = np.nan
print(x, y)

In [ ]:
#ANSWER

pd.isna(x), pd.isna(y)

In [ ]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

In [ ]:
#ANSWER
pd.isna(array)

How is the pandas I/O parameter `na_values` used?

? ANSWER:

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [ ]:
#ANSWER
bikes.season.value_counts()

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns:

In [ ]:
#ANSWER
bikes[['instant','dteday','windspeed']].agg(['min','max'])

Compute and print the overall minimum and maximum of the numeric data columns:

In [ ]:
bikes_min, bikes_max = (min(bikes.min(numeric_only=True)), max(bikes.max(numeric_only=True)))
bikes_min, bikes_max

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [ ]:
bikes['atemp'].quantile(0.5)

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code:

In [ ]:
bikes['atemp'].quantile((0.1, 0.25, 0.5, 0.75, 0.9))

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [ ]:
type(bikes['atemp'])

In [ ]:
bikes.sample(5)

Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indicates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [ ]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)

What is `atemp_level`?

In [ ]:
#ANSWER
print(type(atemp_level))

Here is a random sample of `atemp_level`:

In [ ]:
atemp_level.sample(5)

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [ ]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [ ]:
bikes['atemp_level'] = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
bikes.query('atemp_level == "mild" and season == 2').shape[0]

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initialise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [31]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

a, b, c, d, e

(array([0., 0., 0.]),
 array([[1., 1., 1.]]),
 array([[[4, 6, 5, 7],
         [2, 7, 6, 8],
         [3, 4, 1, 9]],
 
        [[2, 8, 7, 5],
         [5, 2, 2, 2],
         [6, 1, 8, 9]]]),
 array([0, 1, 2, 3]),
 array([[1, 2, 3, 4],
        [5, 6, 7, 8]]))

## Load Data

Load rock.csv and clean the dataset.

In [32]:
df = pd.read_csv('../../../IOD/DATA/rock.csv')

df

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,NaN,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


## Check Column Names

Check column names and clean.

In [33]:
# ANSWER

df.columns

df.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


In [34]:
df.rename(columns={'Song Clean':'song', 'ARTIST CLEAN':'artist', 'Release Year':'release_year', 'COMBINED':'combined', 'First?':'first', 'Year?':'year', 'PlayCount':'play_count'}, inplace=True)

In [35]:
df.drop(columns ='F*G', inplace=True)

## Replace Null Values With 0

Check 'release_year' column whether this column has any null values or not. Replace null values with 0.

In [36]:
# ANSWER

df.isnull().sum()

song              0
artist            0
release_year    577
combined          0
first             0
year              0
play_count        0
dtype: int64

In [37]:
df.release_year.fillna(0, inplace=True)

In [38]:
df.isnull().sum()

song            0
artist          0
release_year    0
combined        0
first           0
year            0
play_count      0
dtype: int64

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column.

In [39]:
# ANSWER
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   song          2230 non-null   object
 1   artist        2230 non-null   object
 2   release_year  2230 non-null   object
 3   combined      2230 non-null   object
 4   first         2230 non-null   int64 
 5   year          2230 non-null   int64 
 6   play_count    2230 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 122.1+ KB


In [40]:
df.release_year.value_counts()

release_year
0                577
1973             104
1977              83
1975              83
1970              81
1971              75
1969              72
1980              70
1978              64
1979              63
1981              61
1967              61
1983              60
1976              56
1982              54
1984              51
1972              50
1974              48
1968              46
1985              39
1987              39
1986              37
1991              34
1989              32
1966              30
1988              29
1965              28
1994              25
1990              22
1993              19
1964              14
1992              14
1999              13
1995              10
1997               9
1963               9
1996               9
1998               6
2002               6
2005               5
2004               5
2012               5
2001               4
2007               3
1962               3
2000               3
2011               3


## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [41]:
# ANSWER
#illogical_rows = df.query('release_year in ["SONGFACTS.COM", "1071"]')
illogical_rows = df['release_year'].isin(['SONGFACTS.COM', '1071']) | df['release_year'] == 0

df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')

df.loc[illogical_rows, 'release_year'] = np.NaN

df['release_year'] = df['release_year'].astype('Int64')

In [42]:
df.release_year.value_counts()

release_year
1973    104
1977     83
1975     83
1970     81
1971     75
1969     72
1980     70
1978     64
1979     63
1967     61
1981     61
1983     60
1976     56
1982     54
1984     51
1972     50
1974     48
1968     46
1985     39
1987     39
1986     37
1991     34
1989     32
1966     30
1988     29
1965     28
1994     25
1990     22
1993     19
1964     14
1992     14
1999     13
1995     10
1996      9
1997      9
1963      9
2002      6
1998      6
2005      5
2004      5
2012      5
2001      4
1962      3
2000      3
2011      3
2003      3
2008      3
2007      3
2013      2
2014      2
2006      1
1958      1
1071      1
1955      1
1961      1
Name: count, dtype: Int64

## Write Some Functions

### Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   song          2230 non-null   object
 1   artist        2230 non-null   object
 2   release_year  1652 non-null   Int64 
 3   combined      2230 non-null   object
 4   first         2230 non-null   int64 
 5   year          2230 non-null   int64 
 6   play_count    2230 non-null   int64 
dtypes: Int64(1), int64(3), object(3)
memory usage: 124.3+ KB


In [44]:
def check_song(row): 
    if row['release_year'] < 1970:
        print(row['song'] + " has a release date < 1970")
    else:
        print(row['song'] + "has a release date >= 1970")

In [45]:
df[df.release_year == 1970]

,song,artist,release_year,combined,first,year,play_count
95,In Memory Of Elizabeth Reed,Allman Brothers Band,1970,In Memory Of Elizabeth Reed by Allman Brothers...,1,1,1
98,Midnight Rider,Allman Brothers Band,1970,Midnight Rider by Allman Brothers Band,1,1,21
142,No Matter What,Badfinger,1970,No Matter What by Badfinger,1,1,2
184,Fairies Wear Boots,Black Sabbath,1970,Fairies Wear Boots by Black Sabbath,1,1,1
187,N.i.b.,Black Sabbath,1970,N.i.b. by Black Sabbath,1,1,8
...,...,...,...,...,...,...,...
2160,CRAZY LOVE,Van Morrison,1970,CRAZY LOVE by Van Morrison,1,1,1
2161,Domino,Van Morrison,1970,Domino by Van Morrison,1,1,1
2163,Into The Mystic,Van Morrison,1970,Into The Mystic by Van Morrison,1,1,1
2165,Moondance,Van Morrison,1970,Moondance by Van Morrison,1,1,14


In [46]:
# ANSWER
check_song(df.iloc[95])

In Memory Of Elizabeth Reedhas a release date >= 1970


### Write a function that converts a column in a DataFrame to a numeric type and otherwise replaces entries with np.nan

In [47]:
# ANSWER

def convert_to_numeric(column):
    column = pd.to_numeric(column, errors='coerce')
    return column

### Apply this last function to your dataset

In [48]:
# ANSWER

convert_to_numeric(df.first)

<bound method NDFrame.first of                                              song       artist  release_year  \
0                                Caught Up in You  .38 Special          1982   
1                                    Fantasy Girl  .38 Special          <NA>   
2                                 Hold On Loosely  .38 Special          1981   
3                          Rockin' Into the Night  .38 Special          1980   
4                               Art For Arts Sake         10cc          1975   
...                                           ...          ...           ...   
2225                      She Loves My Automobile       ZZ Top          <NA>   
2226                            Tube Snake Boogie       ZZ Top          1981   
2227                                         Tush       ZZ Top          1975   
2228                                   TV Dinners       ZZ Top          1983   
2229  WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO       ZZ Top          1973   

        

### 'Describe' the new DataFrame

In [49]:
# ANSWER
df.describe()

,release_year,first,year,play_count
count,1652.0,2230.0,2230.000000,2230.000000
mean,1978.019976,1.0,0.741256,16.872646
std,24.191247,0.0,0.438043,25.302972
min,1071.0,1.0,0.000000,0.000000
25%,1971.0,1.0,0.000000,1.000000
50%,1977.0,1.0,1.000000,4.000000
75%,1984.0,1.0,1.000000,21.000000
max,2014.0,1.0,1.000000,142.000000


>


In [50]:
df.release_year.value_counts()

release_year
1973    104
1977     83
1975     83
1970     81
1971     75
1969     72
1980     70
1978     64
1979     63
1967     61
1981     61
1983     60
1976     56
1982     54
1984     51
1972     50
1974     48
1968     46
1985     39
1987     39
1986     37
1991     34
1989     32
1966     30
1988     29
1965     28
1994     25
1990     22
1993     19
1964     14
1992     14
1999     13
1995     10
1996      9
1997      9
1963      9
2002      6
1998      6
2005      5
2004      5
2012      5
2001      4
1962      3
2000      3
2011      3
2003      3
2008      3
2007      3
2013      2
2014      2
2006      1
1958      1
1071      1
1955      1
1961      1
Name: count, dtype: Int64



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



